MPT-7B
----

# Setup

In [ ]:
!pip install transformers einops 
# !pip install xformers 
# !pip instal triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python
! nvidia-smi

In [ ]:
import transformers
import torch
from google.colab import drive
google_drive = '/content/drive'
drive.mount(google_drive)

# Model loading and config

In [ ]:
model_dir = google_drive + '/My Drive/coding/mpt7b'
# name = 'mosaicml/mpt-7b'
name = model_dir

config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
# config.attn_config['attn_impl'] = 'triton' # haven't been able to get this to work
config.init_device = 'cuda:0' # needs to be defined here or below to use GPU
# config.max_seq_len = 4096 # (input + output) tokens can now be up to 4096
# dtype = torch.bfloat16 # Load model weights in bfloat16 -- requires A100 GPUs
# dtype = torch.float32 # requires much more than 16 gb of GPU ram
dtype = torch.float16  # seems to work with triton disabled, so runnable on V100/T4

model = transformers.AutoModelForCausalLM.from_pretrained(
  name,
  config=config,
  torch_dtype=dtype,
  trust_remote_code=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

# Text generation

In [ ]:
pipe = transformers.pipeline('text-generation', model=model, tokenizer=tokenizer, device='cuda:0')

prompt = 'Here is a recipe for vegan banana bread:\n'
# prompt = "Hong Kong is a city in southern China. It is a major center of trade and finance. It was governed as a crown colony by the Briths from 1841 to 1997. Beginning in 1997 "

with torch.autocast('cuda', dtype=dtype):
    print(
        pipe(prompt,
            max_new_tokens=100,
            do_sample=True,
            use_cache=True))

# Saving locally

In [ ]:
save_path = model_dir
config.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)